# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder: OPTIONAL to execute C++ code</h2>
            <span style="color:#f71;">As an alternative, you can run it on the website given yesterday</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">Important Note</h1>
            <span style="color:#900;">
            In this lab, I use free open source models on Ollama. I also use paid open-source models via Groq and OpenRouter. Only pick the models you want to!
            </span>
        </td>
    </tr>
</table>

In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess


In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:6]}")
else:
    print("OpenRouter API Key not set (and this is optional)")



OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set (and this is optional)
Google API Key exists and begins AI
Grok API Key not set (and this is optional)
Groq API Key not set (and this is optional)
OpenRouter API Key not set (and this is optional)


In [20]:
# Connect to client libraries

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
grok_url = "https://api.x.ai/v1"
groq_url = "https://api.groq.com/openai/v1"
#local
import requests
requests.get("http://localhost:11434").content

OLLAMA_BASE_URL = "http://localhost:11434/v1"


openrouter_url = "https://openrouter.ai/api/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
openrouter = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)


In [21]:
models = ["gpt-5-nano", "claude-sonnet-4-5-20250929", "grok-4", "gemini-2.5-flash-lite", "qwen2.5", "deepseek", "llama3.2", "qwen/qwen3-coder-30b-a3b-instruct", "openai/gpt-oss-120b", ]

clients = {"gpt-5-nano": openai, "claude-sonnet-4-5-20250929": anthropic, "grok-4": grok, "gemini-2.5-flash-lite": gemini, "openai/gpt-oss-120b": groq, "qwen2.5": ollama, "deepseek": ollama, "llama3.2": ollama, "qwen/qwen3-coder-30b-a3b-instruct": openrouter}

# Want to keep costs ultra-low? Replace this with models of your choice, using the examples from yesterday

In [5]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

{'os': {'system': 'Windows',
  'arch': 'AMD64',
  'release': '10',
  'version': '10.0.19045',
  'kernel': '10',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'x86_64-w64-mingw32'},
 'package_managers': ['winget'],
 'cpu': {'brand': 'Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz',
  'cores_logical': 8,
  'cores_physical': 4,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'gcc.EXE (Rev2, Built by MSYS2 project) 14.2.0',
   'g++': 'g++.EXE (Rev2, Built by MSYS2 project) 14.2.0',
   'clang': '',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': ''},
  'linkers': {'ld_lld': ''}}}

## Overwrite this with the commands from yesterday

Or just use the website like yesterday:

 https://www.programiz.com/cpp-programming/online-compiler/

In [6]:
compile_command = ["clang++", "-std=c++17", "-Ofast", "-mcpu=native", "-flto=thin", "-fvisibility=hidden", "-DNDEBUG", "main.cpp", "-o", "main"]
run_command = ["./main"]


## And now, on with the main task

In [7]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [8]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [9]:
def write_output(cpp):
    with open("main.cpp", "w") as f:
        f.write(cpp)

In [10]:
def port(model, python):
    client = clients[model]
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)
    return reply

In [11]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [12]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [13]:
def compile_and_run():
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred:\n{e.stderr}")

In [22]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=28, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=28)
    with gr.Row():
        model = gr.Dropdown(models, label="Select model", value=models[0])
        convert = gr.Button("Convert code")

    convert.click(port, inputs=[model, python], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


# Gemini flash 2.5 lite
# code
```cpp
#include <iostream>
#include <vector>
#include <cmath>
#include <iomanip>
#include <chrono>

// Function to calculate the result
double calculate(int iterations, double param1, double param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        // Calculate j = i * param1 - param2
        double j1 = static_cast<double>(i) * param1 - param2;
        // Update result: result -= (1/j1)
        result -= (1.0 / j1);

        // Calculate j = i * param1 + param2
        double j2 = static_cast<double>(i) * param1 + param2;
        // Update result: result += (1/j2)
        result += (1.0 / j2);
    }
    return result;
}

int main() {
    // Record start time
    auto start_time = std::chrono::high_resolution_clock::now();

    // Define parameters and call the calculate function
    int iterations = 200000000;
    double param1 = 4.0;
    double param2 = 1.0;
    double final_result = calculate(iterations, param1, param2) * 4.0;

    // Record end time
    auto end_time = std::chrono::high_resolution_clock::now();

    // Calculate execution duration
    std::chrono::duration<double> elapsed = end_time - start_time;

    // Print the result with specified precision
    std::cout << std::fixed << std::setprecision(12) << "Result: " << final_result << std::endl;
    // Print the execution time with specified precision
    std::cout << std::fixed << std::setprecision(6) << "Execution Time: " << elapsed.count() << " seconds" << std::endl;

    return 0;
}
```
## Result
Result: 3.141592656089

Execution Time: 2.046798 seconds


## gpt 5 -nano
```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

static inline double calculate_fast(long long iterations, double p1, double p2) {
    double result = 1.0;
    double di = p1; // i = 1 => i*p1
    long long i = 1;
    long long limit = iterations - (iterations % 4);

    for (; i <= limit; i += 4) {
        // i = 1
        double j1 = di - p2;
        result -= 1.0 / j1;
        double j2 = di + p2;
        result += 1.0 / j2;
        di += p1;

        // i = 2
        j1 = di - p2;
        result -= 1.0 / j1;
        j2 = di + p2;
        result += 1.0 / j2;
        di += p1;

        // i = 3
        j1 = di - p2;
        result -= 1.0 / j1;
        j2 = di + p2;
        result += 1.0 / j2;
        di += p1;

        // i = 4
        j1 = di - p2;
        result -= 1.0 / j1;
        j2 = di + p2;
        result += 1.0 / j2;
        di += p1;
    }

    for (; i <= iterations; ++i) {
        double j1 = di - p2;
        result -= 1.0 / j1;
        double j2 = di + p2;
        result += 1.0 / j2;
        di += p1;
    }

    return result;
}

int main() {
    const long long iterations = 200000000LL;
    const double p1 = 4.0;
    const double p2 = 1.0;

    auto t0 = std::chrono::high_resolution_clock::now();
    double result = calculate_fast(iterations, p1, p2) * 4.0;
    auto t1 = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = t1 - t0;

    std::cout.setf(std::ios::fixed);
    std::cout << std::setprecision(12);
    std::cout << "Result: " << result << "\n";
    std::cout << std::setprecision(6);
    std::cout << "Execution Time: " << elapsed.count() << " seconds\n";

    return 0;
}
```

Result: 3.141592656089

Execution Time: 3.203772 seconds


=== Code Execution Successful ===

In [ ]:
compile_and_run()

# llama.2 execute but wrong code
```cpp
#include <iostream>
#include <chrono>

void calculate(float iterations, float param1, float param2) {
    float result = 0.0;
    for (unsigned int i = 1; i <= iterations; ++i) {
        float j = i * param1 - param2;
        // Avoid division by zero
        if (!j)
            continue;
        result -= 1 / j;
        j = i * param1 + param2;
        if (!j)
            continue;
        result += 1 / j;
    }
    // Calculate and print result
    result *= 4;
    std::cout << "Result: " << result << std::endl;
}

int main() {
    // Measure execution time
    auto start_time = std::chrono::high_resolution_clock::now();
    calculate(200000000, 4, 1);
    auto end_time = std::chrono::high_resolution_clock::now();

    // Print measuremnet results 
    std::cout << "Execution Time: ";
    auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(end_time - start_time).count();
    std::cout << (double)duration / 1000.0 << " seconds" << std::endl;

    return 0;
}
```
Result: -0.858405
Execution Time: 1.538 seconds


=== Code Execution Successful ===

Qwen 2.5 Coder: Fail  
DeepSeek Coder v2: 0.114050084  
OpenAI gpt-oss 20B: 0.080438  
Qwen 30B: 0.113734  
OpenAI gpt-oss 120B: 1.407383




In Ed's experiments, the performance speedups were:

9th place: Qwen 2.5 Coder: Fail  
8th place: OpenAI GPT-OSS 120B: 14X speedup    
7th place: DeepSeek Coder v2: 168X speedup  
6th place: Qwen3 Coder 30B: 168X speedup   
5th place: Claude Sonnet 4.5: 184X speedup   
4th place: GPT-5: 233X speedup  
**3rd place: oss-20B: 238X speedup**  
2nd place: Grok 4: 1060X speedup  
1st place: Gemini 2.5 Pro: 1440X speedup  